# Parallel/efficient coding in Python

Python (more presicely: the [CPython](https://en.wikipedia.org/wiki/CPython) interpreter) uses a [Global interpreter lock (GIL)](https://en.wikipedia.org/wiki/Global_Interpreter_Lock) which prohibits simultaneous execution of Python code.

## Overview

In this tutorial, we'll cover these common problems/situations/solutions:

- If your code is I/O bound, [threads](https://docs.python.org/2/library/thread.html) is a simple solution
- If your code is CPU bound, [multiprocessing](https://docs.python.org/2/library/multiprocessing.html) is the way to go
- If your code can be vectorised: use [numpy](http://www.numpy.org)
- If your code is still too slow: write your own C extensions with [cython](http://www.cython.org)

This list is by no means complete and is just a starting point for getting you into parallel/efficient coding with Python. There are plenty of other very nice solutions for all kinds of problems out there, but this is beyond the scope of this tutorial.

If you are not familiar with Python, I'd suggest [learning it the (not so) hard way](http://learnpythonthehardway.org). Or continue reading, it describes very briefly the needed basics.

## A very short Python primer

This is how we define functions:

In [2]:
def function(argument):
    """
    This function just prints the given argument.
    
    :param argument: what to print
    :returns:        also returns the argument
    
    """
    print("Calling function with argument %s" % argument)
    return argument

In [3]:
function(42)

Calling function with argument 42


42

This is how we define classes:

In [4]:
class Class(object):
    """A simple meaningless Class"""
    def __init__(self, attribute):
        """
        This method get's called during object initialisation.
        
        :param attribute: attribute of the class to save
        
        """
        self.attribute = attribute
    
    def tell(self):
        """Tell the world about us."""
        print("I'm a Class with a very nice attribute: %s" % self.attribute)

In [5]:
A = Class('smart')
A.tell()

I'm a Class with a very nice attribute: smart


Since all (our) functions and classes are well documented, we can always ask how to use them:

In [6]:
function?

In [7]:
Class?

Python also has lists which can be accessed by index (indices always start at 0):

In [8]:
x = [1, 2, 4.5, 'bla']

In [9]:
x[1]

2

In [10]:
x[3]

'bla'

One of the more fancy stuff we can do in Python is list comprehensions.

Here's a simple example of how to calculate the power of some numbers:

In [11]:
[x ** 2 for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

Ok, that's it for now, let's speed things up :)

## Threads

From the Python `threading` documentation (https://docs.python.org/2/library/threading.html):

> In CPython, due to the Global Interpreter Lock, only one thread can execute Python code at once (even though certain performance-oriented libraries might overcome this limitation). If you want your application to make better use of the computational resources of multi-core machines, you are advised to use `multiprocessing`. However, threading is still an appropriate model if you want to run multiple I/O-bound tasks simultaneously.

Thus, let's move on to `multiprocessing` -- it also has a simpler interface ;)

## Multiprocessing

In [12]:
import multiprocessing as mp

###A simple CPU bound example.

source: http://www.parallelpython.com/content/view/17/31/#SUM_PRIMES

In [13]:
import math

def isprime(n):
    """Returns True if n is prime and False otherwise"""
    if not isinstance(n, int):
        raise TypeError("argument passed to is_prime is not of 'int' type")
    if n < 2:
        return False
    if n == 2:
        return True
    max = int(math.ceil(math.sqrt(n)))
    i = 2
    while i <= max:
        if n % i == 0:
            return False
        i += 1
    return True

def sum_primes(n):
    """Calculates sum of all primes below given integer n"""
    return sum([x for x in xrange(2,n) if isprime(x)])

In [14]:
sum_primes(100)

1060

In [15]:
%timeit sum_primes(100)

10000 loops, best of 3: 77.5 µs per loop


Not bad, why should we speed up something like this?

In [16]:
%timeit sum_primes(10000)

100 loops, best of 3: 17.4 ms per loop


Ok, things get slower the higher the numbers are. What if we do this for a bunch of numbers:

In [17]:
[sum_primes(n) for n in xrange(10)]

[0, 0, 0, 2, 5, 5, 10, 10, 17, 17]

In [18]:
%timeit [sum_primes(n) for n in xrange(1000)]

1 loops, best of 3: 501 ms per loop


Ok, this can definitely be run in parallel, since we are asking for the same thing for a lot of of numbers.

Python has a `map` function, which maps a list of arguments to a single function. 

In [19]:
map(sum_primes, xrange(10))

[0, 0, 0, 2, 5, 5, 10, 10, 17, 17]

This is basically the same as what we had above with our list comprehension.

In [20]:
%timeit map(sum_primes, xrange(1000))

1 loops, best of 3: 502 ms per loop


`multiprocessing` offers the same `map` function in it's `Pool` class. Let's create a Pool with 2 simultaneous threads (i.e. processes).

In [21]:
pool = mp.Pool(2)
pool.map(sum_primes, xrange(10))

[0, 0, 0, 2, 5, 5, 10, 10, 17, 17]

In [22]:
%timeit pool.map(sum_primes, xrange(1000))

1 loops, best of 3: 280 ms per loop


This is a speed-up of almost 2x, which is exactly what we expected (2 cores - some overhead).

## Vectorisation

We can solve a lot of stuff in almost no time if we avoid loops and vectorise the expressions instead. [Numpy](http://www.numpy.org) does an excellent job here!

Some numpy basics:

In [23]:
import numpy as np

We can define arrays simple as that:

In [24]:
x = np.array([1, 2, 5, 15])

In [25]:
x

array([ 1,  2,  5, 15])

As long as it can be casted to an array, we can almost everything as input for an array:

In [26]:
np.array(map(sum_primes, xrange(10)))

array([ 0,  0,  0,  2,  5,  5, 10, 10, 17, 17])

Or we define arrays with some special functions:

In [27]:
np.zeros(10)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [28]:
np.arange(10.)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])

Numpy supports indexing and slicing

In [29]:
x = np.arange(10)

Get a single item of the array:

In [30]:
x[3]

3

Get a slice of the array:

In [31]:
x[1:5]

array([1, 2, 3, 4])

Get everything starting from index 4 to the end:

In [32]:
x[4:]

array([4, 5, 6, 7, 8, 9])

Negative indices are counted backwards from the end.
Get everything before the last element:

In [33]:
x[:-1]

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

### Let's define another problem: [comb filters](https://en.wikipedia.org/wiki/Comb_filter).

Wikipedia says:

> In signal processing, a comb filter adds a delayed version of a signal to itself, causing constructive and destructive interference.

These filters can be either feed forward or backward, depending on wheter the signal itself or the output of the filter is delayed and added to the signal.

Feed forward:

$y[n] = x[n] + \alpha * x[n - \tau]$

Feed backward:

$y[n] = x[n] + \alpha * y[n - \tau]$

In [34]:
def feed_forward_comb_filter_loop(signal, tau, alpha):
    """
    Filter the signal with a feed forward comb filter.

    :param signal: signal
    :param tau:    delay length
    :param alpha:  scaling factor
    :return:       comb filtered signal

    """
    # y[n] = x[n] + α * x[n - τ]
    if tau <= 0:
        raise ValueError('tau must be greater than 0')
    # init the output array
    y = np.zeros(len(signal))
    # add the delayed version of the signal to itself
    for i in range(len(signal)):
        if i < tau:
            y[i] = signal[i]
        else:
            y[i] += alpha * signal[i - tau]
    # return the filtered signal
    return y

In [35]:
feed_forward_comb_filter_loop(np.arange(10.), 4, 0.5)

array([ 0. ,  1. ,  2. ,  3. ,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5])

In [36]:
%timeit feed_forward_comb_filter_loop(np.arange(100000.), 4, 0.5)

10 loops, best of 3: 35.1 ms per loop


In [37]:
def feed_forward_comb_filter(signal, tau, alpha):
    """
    Filter the signal with a feed forward comb filter.

    :param signal: signal
    :param tau:    delay length
    :param alpha:  scaling factor
    :return:       comb filtered signal

    """
    # y[n] = x[n] + α * x[n - τ]
    if tau <= 0:
        raise ValueError('tau must be greater than 0')
    # init the output array as a copy of the input signal, since
    # the output is the input signal plus a delayed version of it
    y = signal.copy()
    # add the delayed signal, starting at index tau
    y[tau:] += alpha * signal[:-tau]
    # return the filtered signal
    return y

In [38]:
%timeit feed_forward_comb_filter(np.arange(100000.), 4, 0.5)

1000 loops, best of 3: 523 µs per loop


This is a nice ~67x speed-up (523 µs vs. 35.1 ms). Let's have a look at the feed backward example:

In [39]:
def feed_backward_comb_filter_loop(signal, tau, alpha):
    """
    Filter the signal with a feed backward comb filter.

    :param signal: signal
    :param tau:    delay length
    :param alpha:  scaling factor
    :return:       comb filtered signal

    """
    # y[n] = x[n] + α * y[n - τ]
    if tau <= 0:
        raise ValueError('tau must be greater than 0')
    # init the output array as a copy of the input signal
    y = signal.copy()
    # loop over the signal, starting at tau
    for n in range(tau, len(signal)):
        # add a delayed version of the output signal
        y[n] = signal[n] + alpha * y[n - tau]
    # return the filtered signal
    return y

In [40]:
%timeit feed_backward_comb_filter_loop(np.arange(100000.), 4, 0.5)

10 loops, best of 3: 38.1 ms per loop


The backward variant has basically the same runtime as the forward version, but unfortunately, we cannot speed this up further with a vectorised expression, since the output depends on the output of a previous step.

And this is where Cython comes in.

## Cython

Some `Cython` basics. First we need to load Cython. To be able to use it within IPython, we need to lad the Cython extension via:

In [41]:
%load_ext Cython

To be able to use Cython code within IPython, we need to add the magic `%%cython` handler as a first line into a cell. The Cython compiler converts everything into C code, compiles it and loads it transparently.

Cython gains most of its tremendous speed gains from static typing.

Let's try with the `isprime` example from above again:

In [42]:
%%cython
# magic cython handler for IPython (must be first line of a cell)

# cython imports use cimport instead of import
cimport cython
# import some C functions to avoid calls to the Python interpreter
from libc.math cimport sqrt, ceil

# define type n as integer
def isprime_cython(int n):
    """Returns True if n is prime and False otherwise"""
    # we can skip the instance check since we have strong typing now
    if n < 2:
        return False
    if n == 2:
        return True
    # define type max as integer
    cdef int max
    # use the C celi & sqrt functions instead of Python's own
    max = int(ceil(sqrt(n)))
    # define type n as integer
    cdef int i = 2
    while i <= max:
        if n % i == 0:
            return False
        i += 1
    return True

def sum_primes_cython(n):
    """Calculates sum of all primes below given integer n"""
    return sum([x for x in xrange(2,n) if isprime_cython(x)])

In [43]:
%timeit sum_primes(10000)
%timeit sum_primes_cython(10000)

100 loops, best of 3: 16.7 ms per loop
1000 loops, best of 3: 677 µs per loop


speed-up ~24x (677 µs vs. 16.7 ms)

In [44]:
%timeit -n 1 [sum_primes(n) for n in xrange(10000)]
%timeit -n 1 [sum_primes_cython(n) for n in xrange(10000)]

1 loops, best of 3: 1min 23s per loop
1 loops, best of 3: 3.13 s per loop


speed-up ~24x (3.13 s vs. 73 s)

In [45]:
%timeit mp.Pool(2).map(sum_primes_cython, (n for n in xrange(10000)))
%timeit mp.Pool(4).map(sum_primes_cython, (n for n in xrange(10000)))

1 loops, best of 3: 1.86 s per loop
1 loops, best of 3: 1.43 s per loop


Starting more threads than physical CPU cores gives some more performance, but does not scale as good because of hyper-threading. Total speed-up is ~50x (1.43s vs. 73s)

In [46]:
%%cython
cimport cython
from libc.math cimport sqrt, ceil

# make this a C function
cdef int isprime_cython_nogil(int n) nogil:
    """Returns True if n is prime and False otherwise"""
    if n < 2:
        return 0
    if n == 2:
        return 1
    cdef int max
    max = int(ceil(sqrt(n)))
    cdef int i = 2
    while i <= max:
        if n % i == 0:
            return 0
        i += 1
    return 1

def sum_primes_cython_nogil(n):
    """Calculates sum of all primes below given integer n"""
    return sum([x for x in xrange(2,n) if isprime_cython_nogil(x)])

In [47]:
%timeit [sum_primes_cython_nogil(n) for n in xrange(10000)]
%timeit mp.Pool(4).map(sum_primes_cython_nogil, (n for n in xrange(10000)))

1 loops, best of 3: 2.54 s per loop
1 loops, best of 3: 1.24 s per loop


A bit faster, because we do not call a Python function from `sum_primes_cython_nogil` any more. Total speed-up is ~66x (1.1s vs. 73s)

These are the Cython basics. Let's apply them to the other example, the backward comb filter which we were not able to vectorise:

In [48]:
%%cython

import numpy as np
cimport cython
cimport numpy as np

def feed_backward_comb_filter(signal, unsigned int tau, float alpha):
    """
    Filter the signal with a feed backward comb filter.

    :param signal: signal
    :param tau:    delay length
    :param alpha:  scaling factor
    :return:       comb filtered signal

    """
    # y[n] = x[n] + α * y[n - τ]
    if tau <= 0:
        raise ValueError('tau must be greater than 0')
    # type definitions
    y = np.copy(signal)
    cdef unsigned int n
    # loop over the complete signal
    for n in range(tau, len(signal)):
        # add a delayed version of the output signal
        y[n] = signal[n] + alpha * y[n - tau]
    # return the filtered signal
    return y

In [49]:
%timeit feed_backward_comb_filter(np.arange(100000.), 4, 0.5)

100 loops, best of 3: 19.5 ms per loop


A bit better (roughly half the time), but still far away from the feed forward variant. Let's see, what kills performance and fix it. Cython has this nice `-a` switch to highlight calls to Python in yellow.

In [50]:
%%cython -a

import numpy as np
cimport cython
cimport numpy as np

def feed_backward_comb_filter(signal, unsigned int tau, float alpha):
    """
    Filter the signal with a feed backward comb filter.

    :param signal: signal
    :param tau:    delay length
    :param alpha:  scaling factor
    :return:       comb filtered signal

    """
    # y[n] = x[n] + α * y[n - τ]
    if tau <= 0:
        raise ValueError('tau must be greater than 0')
    # type definitions
    y = np.copy(signal)
    cdef unsigned int n
    # loop over the complete signal
    for n in range(tau, len(signal)):
        # add a delayed version of the output signal
        y[n] = signal[n] + alpha * y[n - tau]
    # return the filtered signal
    return y

In line 25, we still have calls to Python within the loop (e.g. `PyNumber_Multiply` and `PyNumber_Add`). We can get rid of it by statically typing the signal as well. Unfortunately, we lose the ability to call the filter function with a signal of arbitrary dimensions.

In [51]:
%%cython

import numpy as np
cimport cython
cimport numpy as np

def feed_backward_comb_filter_1d(np.ndarray[np.float_t, ndim=1] signal,
                                 unsigned int tau,
                                 float alpha):
    """
    Filter the signal with a feed backward comb filter.

    :param signal: signal
    :param tau:    delay length
    :param alpha:  scaling factor
    :return:       comb filtered signal

    """
    # y[n] = x[n] + α * y[n - τ]
    if tau <= 0:
        raise ValueError('tau must be greater than 0')
    # type definitions
    cdef np.ndarray[np.float_t, ndim=1] y = np.copy(signal)
    cdef unsigned int n
    # loop over the complete signal
    for n in range(tau, len(signal)):
        # add a delayed version of the output signal
        y[n] = signal[n] + alpha * y[n - tau]
    # return the filtered signal
    return y

In [52]:
%timeit feed_backward_comb_filter_1d(np.arange(100000.), 4, 0.5)

1000 loops, best of 3: 546 µs per loop


Much better, let's check again.

In [53]:
%%cython -a

import numpy as np
cimport cython
cimport numpy as np

def feed_backward_comb_filter_1d(np.ndarray[np.float_t, ndim=1] signal,
                                 unsigned int tau,
                                 float alpha):
    """
    Filter the signal with a feed backward comb filter.

    :param signal: signal
    :param tau:    delay length
    :param alpha:  scaling factor
    :return:       comb filtered signal

    """
    # y[n] = x[n] + α * y[n - τ]
    if tau <= 0:
        raise ValueError('tau must be greater than 0')
    # type definitions
    cdef np.ndarray[np.float_t, ndim=1] y = np.copy(signal)
    cdef unsigned int n
    # loop over the complete signal
    for n in range(tau, len(signal)):
        # add a delayed version of the output signal
        y[n] = signal[n] + alpha * y[n - tau]
    # return the filtered signal
    return y

For the sake of completeness, let's get rid of these `Pyx_RaiseBufferIndexError` in line 27 as well. We tell Cython that it does not need to check for bounds by adding a `@cython.boundscheck(False)` decorator.

In [54]:
%%cython

import numpy as np
cimport cython
cimport numpy as np

@cython.boundscheck(False)
def feed_backward_comb_filter_1d(np.ndarray[np.float_t, ndim=1] signal,
                                 unsigned int tau,
                                 float alpha):
    """
    Filter the signal with a feed backward comb filter.

    :param signal: signal
    :param tau:    delay length
    :param alpha:  scaling factor
    :return:       comb filtered signal

    """
    # y[n] = x[n] + α * y[n - τ]
    if tau <= 0:
        raise ValueError('tau must be greater than 0')
    # type definitions
    cdef np.ndarray[np.float_t, ndim=1] y = np.copy(signal)
    cdef unsigned int n
    # loop over the complete signal
    for n in range(tau, len(signal)):
        # add a delayed version of the output signal
        y[n] = signal[n] + alpha * y[n - tau]
    # return the filtered signal
    return y

In [55]:
%timeit feed_backward_comb_filter_1d(np.arange(100000.), 4, 0.5)

1000 loops, best of 3: 512 µs per loop


To get back the flexibility of the Python/Numpy solution we need to define a wrapper function:

In [56]:
%%cython

import numpy as np

cimport cython
cimport numpy as np

def feed_backward_comb_filter(signal, tau, alpha):
    """
    Filter the signal with a feed backward comb filter.

    :param signal: signal
    :param tau:    delay length
    :param alpha:  scaling factor
    :return:       comb filtered signal

    """
    if signal.ndim == 1:
        return feed_backward_comb_filter_1d(signal, tau, alpha)
    elif signal.ndim == 2:
        return feed_backward_comb_filter_2d(signal, tau, alpha)
    else:
        raise ValueError('signal must be 1d or 2d')

@cython.boundscheck(False)
def feed_backward_comb_filter_1d(np.ndarray[np.float_t, ndim=1] signal,
                                 unsigned int tau,
                                 float alpha):
    """
    Filter the signal with a feed backward comb filter.

    :param signal: signal
    :param tau:    delay length
    :param alpha:  scaling factor
    :return:       comb filtered signal

    """
    # y[n] = x[n] + α * y[n - τ]
    if tau <= 0:
        raise ValueError('tau must be greater than 0')
    # type definitions
    cdef np.ndarray[np.float_t, ndim=1] y = np.copy(signal)
    cdef unsigned int n
    # loop over the complete signal
    for n in range(tau, len(signal)):
        # add a delayed version of the output signal
        y[n] = signal[n] + alpha * y[n - tau]
    # return the filtered signal
    return y

@cython.boundscheck(False)
def feed_backward_comb_filter_2d(np.ndarray[np.float_t, ndim=2] signal,
                                 unsigned int tau,
                                 float alpha):
    """
    Filter the signal with a feed backward comb filter.

    :param signal: signal
    :param tau:    delay length
    :param alpha:  scaling factor
    :return:       comb filtered signal

    """
    # y[n] = x[n] + α * y[n - τ]
    if tau <= 0:
        raise ValueError('tau must be greater than 0')
    # type definitions
    cdef np.ndarray[np.float_t, ndim=2] y = np.copy(signal)
    cdef unsigned int d, n
    # loop over the dimensions
    for d in range(2):
        # loop over the complete signal
        for n in range(tau, len(signal)):
            # add a delayed version of the output signal
            y[n, d] = signal[n, d] + alpha * y[n - tau, d]
    # return
    return y


In [57]:
%timeit feed_backward_comb_filter(np.arange(100000.), 4, 0.5)
%timeit feed_backward_comb_filter(np.arange(100000.).reshape(-1, 2), 4, 0.5)

1000 loops, best of 3: 522 µs per loop
1000 loops, best of 3: 530 µs per loop
